# Top End Assembly Vibration Study

As described in [OBS-467], we have been noticing strong vibrations on the Top End Assembly (TEA) of the Simonyi Survey Telescope (SST). 
The first time we saw these vibrations was on March 9th (Saturday) and on March 10th (Sunday). 

Accordingly to [SITCOM-1285], on March 15th (Friday), we ran soak tests using the Rotator at different configurations. 
We want to investigate any possible vibrations during that period to confirm the hypothesis that the Rotator could be causing this vibration.

In addition to both cases, on Marth 21th (Thursday), we executed [BLOCK-197] a couple of times. 
[OBS-498] reports multiple faults on the hexapods associated with the compensation mode. 
It also indicates that we might have vibrations during its execution, we need to review it. 

Finally, on March 25th, we executed [BLOCK-197] twice. 
First with the rotator in ENABLED. 
Second with the rotator in STANDBY and with its cabinet turned off. 
We want to compare both cases. 

[BLOCK-197]: https://rubinobs.atlassian.net/browse/BLOCK-197
[OBS-467]: https://rubinobs.atlassian.net/browse/OBS-467
[OBS-498]: https://rubinobs.atlassian.net/browse/OBS-498
[SITCOM-1285]: https://rubinobs.atlassian.net/browse/SITCOM-1285

## Notebook Preparation

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import time
import warnings

from argparse import Namespace
from astropy.time import Time, TimeDelta
from scipy.fft import fft, fftfreq
from scipy.signal import detrend, get_window, welch
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst_efd_client.efd_helper import merge_packed_time_series


# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

# Create an EFD client
client = makeEfdClient()

# Global variables definition - SST = Simonyi Survey Telescope
sensorNames = [
    "SST top end ring +x -y",
    "SST top end ring -x -y",
    "SST spider spindle",
    "SST M2 surrogate",
]
sensorNamesShort = ["ter_pxmy", "ter_mxmy", "spider", "m2surr"]

# Create a folder for plots
os.makedirs("./plots", exist_ok=True)

## OBS-467 - The initial problem

In [ ]:
# Whole weekend
start = Time("2024-03-09T10:00:00Z", scale="utc", format="isot")
end = Time("2024-03-11T06:00:00Z", scale="utc", format="isot")

# # Zoom on strong vibrations
# start = Time("2024-03-10T18:00:00Z", scale="utc", format="isot")
# end = Time("2024-03-11T03:00:00Z", scale="utc", format="isot")

# Zoom on weaker vibrations
# start = Time("2024-03-10T03:00:00Z", scale="utc", format="isot")
# end = Time("2024-03-10T03:02:00Z", scale="utc", format="isot")

df_az = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    columns=["actualPosition", "actualTorque"],
    begin=start,
    end=end,
)

df_el = getEfdData(
    client,
    "lsst.sal.MTMount.elevation",
    columns=["actualPosition", "actualTorque"],
    begin=start,
    end=end,
)

df_rot = getEfdData(
    client,
    "lsst.sal.MTRotator.rotation",
    columns=["actualPosition"],
    begin=start,
    end=end,
)

df_rot_t = getEfdData(
    client,
    "lsst.sal.MTRotator.motors",
    columns="*",
    begin=start,
    end=end,
)

df_m2 = getEfdData(
    client,
    "lsst.sal.MTM2.positionIMS",
    columns=["x", "y", "z"],
    begin=start,
    end=end,
)

df_hex = getEfdData(
    client,
    "lsst.sal.MTHexapod.application",
    columns=[f"position{i}" for i in range(3)] + ["salIndex"],
    begin=start,
    end=end,
)

df_m2r = df_m2.rolling("1S").mean()
df_camhex = df_hex[df_hex.salIndex == 1]
df_m2hex = df_hex[df_hex.salIndex == 2]

<div class="alert alert-warning">
The plot below takes at least 2.5 min to be executed. 
</div>

In [ ]:
%matplotlib inline
fig, (
    ax_m2,
    ax_m2r,
    ax_camhex,
    ax_m2hex,
    ax_rot,
    ax_rot_torque,
    ax_azp,
    ax_elp,
    ax_azt,
    ax_elt,
) = plt.subplots(nrows=10, figsize=(16, 16), sharex=True)

ax_m2.plot(df_m2.x, label="x")
ax_m2.plot(df_m2.y, label="y")
ax_m2.plot(df_m2.z, label="z")
ax_m2.grid(":", alpha=0.25)
ax_m2.set_ylabel("M2 IMS\n Position [um]")
ax_m2.legend(loc="upper left")

ax_m2r.plot(df_m2.x - df_m2r.x, label="x")
ax_m2r.plot(df_m2.y - df_m2r.y, label="y")
ax_m2r.plot(df_m2.z - df_m2r.z, label="z")
ax_m2r.grid(":", alpha=0.25)
ax_m2r.set_ylabel("M2 IMS Position\n Minus Rolling Average [um]")
ax_m2r.legend(loc="upper left")

ax_camhex.plot(df_camhex.position0, label="x")
ax_camhex.plot(df_camhex.position1, label="y")
ax_camhex.plot(df_camhex.position2, label="z")
ax_camhex.grid(":", alpha=0.25)
ax_camhex.set_ylabel("CamHex\n Position [um]")
ax_camhex.legend(loc="upper left")

ax_m2hex.plot(df_m2hex.position0, label="x")
ax_m2hex.plot(df_m2hex.position1, label="y")
ax_m2hex.plot(df_m2hex.position2, label="z")
ax_m2hex.grid(":", alpha=0.25)
ax_m2hex.set_ylabel("M2Hex\n Position [um]")
ax_m2hex.legend(loc="upper left")

ax_azp.plot(df_az.actualPosition, label="Azimuth")
ax_azp.grid(":", alpha=0.25)
ax_azp.set_ylabel("Azimuth\n Actual Position\n [deg]")
ax_azp.legend(loc="upper left")

ax_elp.plot(df_el.actualPosition, label="Elevation")
ax_elp.grid(":", alpha=0.25)
ax_elp.set_ylabel("Elevation\n Actual Position\n [deg]")
ax_elp.legend(loc="upper left")

ax_azt.plot(df_az.actualTorque, label="Azimuth")
ax_azt.grid(":", alpha=0.25)
ax_azt.set_ylabel("Azimuth\n Actual Torque\n [N.m]")
ax_azt.legend(loc="upper left")

ax_elt.plot(df_el.actualTorque, label="Elevation")
ax_elt.grid(":", alpha=0.25)
ax_elt.set_ylabel("Elevation\n Actual Torque\n [N.m]")
ax_elt.legend(loc="upper left")

ax_rot_torque.plot(df_rot_t.torque0 - df_rot_t.torque0.mean(), label="Motor 1")
ax_rot_torque.plot(df_rot_t.torque1 - df_rot_t.torque1.mean(), label="Motor 2")
ax_rot_torque.grid(":", alpha=0.25)
ax_rot_torque.set_ylabel("Rotator\n Actual Torque\n [N.m]")
ax_rot_torque.legend(loc="upper left")

ax_rot.plot(df_rot.actualPosition * 1e3, label="Actual Position")
ax_rot.grid(":", alpha=0.25)
ax_rot.set_ylabel("Rotator Angle\n [$10^-3$ deg]")
ax_rot.set_xlim(df_rot.index[0], df_rot.index[-1])
ax_rot.set_xlabel("Time [utc]")
ax_rot.legend(loc="upper left")


fig.suptitle(
    "OBS-467 Investigation - M2/Rotator/Hexapods Vibrrations\n"
    f"From {start.to_value('iso', subfmt='date_hm')} to {end.to_value('iso', subfmt='date_hm')}"
)

fig.autofmt_xdate()
fig.tight_layout()

fig.savefig("./plots/obs467_investigation_whole_weekend.png")
plt.show()

The plots above show a low amplitude vibration on the Rotator and on M2 from 00:40 to 15:00 UTC on March 10th.  
Following the logs in OLE, the first problematic measurements using the Laser Tracker started to appear around 00:44. 
This is consistent with what we see above. 

We checked the vibrations on the rotator actual position on Chronograf, we found that the exact time that it started was at `00:37:55`.
At `14:41:13`, the amplitude of these oscillations are smaller and last until `16:08:41`.  

Alysha confirmed that all the vibrations/oscillations after 17h UTC come from other tests and are unrelated to the sounds we hear when going to Level 8. 

Another important aspect to consider are the amplitudes of the vibrations. 
A quick eye inspection show that the oscillation on the rotator have an amplitude of about 0.1 x 10^-3 degrees. 
The M2 have oscillations around 1 um. 
None of these two justify the 2 mm displacement that we were measuring with the Laser Tracker.

When looking at the hexapods data, we can see the warm-up sequence twice. 
First, at around 18h UTC on March 9th and, second, near 14h UTC on March 10th. 
These two data sets give us an example of the range of movement on the hexapods. 

We can also see both hexapods being exercised quite a lot near the time when the vibrations/oscillations started. 

Several questions remain for this time window and we can use the following events/commands to investigate a bit more:
- summary states for the hexapods and the rotator
- drive states for the hexapods and the rotator
- when was the compensation mode enabled/disabled?

## Helper Functions

Let me save all the functions used in this notebook within this session.  
This makes navigation easier. 

### query_efd_data

In [ ]:
def query_efd_data(start, end):
    """
    Query the following topics from the EFD:
    - lsst.sal.MTRotator.rotation
    - lsst.sal.MTM2.positionIMS
    - lsst.sal.MTHexapod.application
    - lsst.sal.ESS.accelerometer

    Parameters
    ----------
    start : astropy.time.Time
        Start of the time series
    end : astropy.time.Time
        End of the time series

    Returns
    -------
    data : Namespace
        An object that contains a data frame for each dataset as attributes.
    """
    data = {}

    data["start"] = start
    data["end"] = end

    data["rot"] = getEfdData(
        client,
        "lsst.sal.MTRotator.rotation",
        columns=["actualPosition"],
        begin=start,
        end=end,
    )

    # Find out if the rotator was on or off based on the size of the dataset
    #   The 5000 is a guess of a tolerance.
    #   If the rotator is on, the size of the dataset it much higher.
    data["rot_status"] = "off" if data["rot"].size < 5000 else "on"

    data["m2"] = getEfdData(
        client,
        "lsst.sal.MTM2.positionIMS",
        columns=["x", "y", "z"],
        begin=start,
        end=end,
    )
    data["m2"] = data["m2"] - data["m2"].mean()
    data["m2"]["times"] = data["m2"].index

    data["hexs"] = getEfdData(
        client,
        "lsst.sal.MTHexapod.application",
        columns=[f"position{i}" for i in range(3)] + ["salIndex"],
        begin=start,
        end=end,
    )
    data["camhex"] = data["hexs"][data["hexs"]["salIndex"] == 1]
    data["m2hex"] = data["hexs"][data["hexs"]["salIndex"] == 2]

    packed_ess_df = getEfdData(
        client,
        "lsst.sal.ESS.accelerometer",
        columns="*",
        begin=start,
        end=end,
    )

    for short_name, long_name in zip(sensorNamesShort, sensorNames):
        fields_df = {}
        sub_df = packed_ess_df.loc[packed_ess_df.sensorName == long_name]

        baseFields = ["accelerationX", "accelerationY", "accelerationZ"]

        for base_field in baseFields:
            fields_df[base_field] = merge_packed_time_series(
                sub_df,
                base_field,
                stride=1,
                ref_timestamp_col="timestamp",
                fmt="unix_tai",
                scale="tai",
            )
            fields_df[base_field] = fields_df[base_field].sort_index()

        sub_df = pd.merge_asof(
            pd.merge_asof(
                fields_df[baseFields[0]],
                fields_df[baseFields[1]],
                left_index=True,
                right_index=True,
            ),
            fields_df[baseFields[2]],
            left_index=True,
            right_index=True,
        )
        sub_df = sub_df - sub_df.mean()
        data[short_name] = sub_df

    return data

### plot_dataframes

In [ ]:
def plot_dataframes(data, title=None, filename=None):
    fig, (
        ax_m2,
        ax_camhex,
        ax_m2hex,
        ax_rot,
        ax_spider,
        ax_m2surr,
        ax_ter_pxmy,
        ax_ter_mxmy,
    ) = plt.subplots(nrows=8, figsize=(16, 16), sharex=True, num=filename)

    ax_m2.plot(data.m2.x, label="x")
    ax_m2.plot(data.m2.y, label="y")
    ax_m2.plot(data.m2.z, label="z")
    ax_m2.grid(":", alpha=0.25)
    ax_m2.set_xlim(data.start.to_datetime(), data.end.to_datetime())
    ax_m2.set_ylabel("M2 IMS\n Position [um]")
    ax_m2.legend(loc="upper left")

    ax_camhex.plot(data.camhex.position0, label="x")
    ax_camhex.plot(data.camhex.position1, label="y")
    ax_camhex.plot(data.camhex.position2, label="z")
    ax_camhex.grid(":", alpha=0.25)
    ax_camhex.set_ylabel("CamHex\n Position [um]")
    ax_camhex.legend(loc="upper left")

    ax_m2hex.plot(data.m2hex.position0, label="x")
    ax_m2hex.plot(data.m2hex.position1, label="y")
    ax_m2hex.plot(data.m2hex.position2, label="z")
    ax_m2hex.grid(":", alpha=0.25)
    ax_m2hex.set_ylabel("M2\n Position [um]")
    ax_m2hex.legend(loc="upper left")

    try:
        ax_rot.plot(data.rot.actualPosition * 1e3, label="Actual Position")
    except (AttributeError, IndexError):
        pass

    ax_rot.grid(":", alpha=0.25)
    ax_rot.set_ylabel("Rotator Angle\n [$10^-3$ deg]")
    ax_rot.set_xlabel("Time [utc]")
    ax_rot.legend(loc="upper left")

    ax_spider.plot(data.spider.accelerationX, label="x", alpha=0.3)
    ax_spider.plot(data.spider.accelerationY, label="y", alpha=0.3)
    ax_spider.plot(data.spider.accelerationZ, label="z", alpha=0.3)
    ax_spider.grid(":", alpha=0.25)
    ax_spider.set_ylabel("SST Spider\nAcceleration [$m.s^{-2}$]")
    ax_spider.legend(loc="upper left")

    ax_m2surr.plot(data.m2surr.accelerationX, label="x", alpha=0.3)
    ax_m2surr.plot(data.m2surr.accelerationY, label="y", alpha=0.3)
    ax_m2surr.plot(data.m2surr.accelerationZ, label="z", alpha=0.3)
    ax_m2surr.grid(":", alpha=0.25)
    ax_m2surr.set_ylabel("SST M2 Surrogate\nAcceleration [$m.s^{-2}$]")
    ax_m2surr.legend(loc="upper left")

    ax_ter_mxmy.plot(data.ter_mxmy.accelerationX, label="x", alpha=0.3)
    ax_ter_mxmy.plot(data.ter_mxmy.accelerationY, label="y", alpha=0.3)
    ax_ter_mxmy.plot(data.ter_mxmy.accelerationZ, label="z", alpha=0.3)
    ax_ter_mxmy.grid(":", alpha=0.25)
    ax_ter_mxmy.set_ylabel("SST top end ring -x -y\nAcceleration [$m.s^{-2}$]")
    ax_ter_mxmy.legend(loc="upper left")

    ax_ter_pxmy.plot(data.ter_pxmy.accelerationX, label="x", alpha=0.3)
    ax_ter_pxmy.plot(data.ter_pxmy.accelerationY, label="y", alpha=0.3)
    ax_ter_pxmy.plot(data.ter_pxmy.accelerationZ, label="z", alpha=0.3)
    ax_ter_pxmy.grid(":", alpha=0.25)
    ax_ter_pxmy.set_ylabel("SST top end ring +x -y\nAcceleration [$m.s^{-2}$]")
    ax_ter_pxmy.legend(loc="upper left")

    if title:
        fig.suptitle(
            f"{title}\n"
            f"From {start.to_value('iso', subfmt='date_hm')} to {end.to_value('iso', subfmt='date_hm')}"
        )

    fig.autofmt_xdate()
    fig.tight_layout()

    if filename:
        fig.savefig(f"./plots/{filename}")

    plt.show()

### calculate_amplitudes

In [ ]:
def calculate_amplitudes(data):
    # Initialize an empty list to hold the data
    new_data = []

    # Iterate over the sensor names and dataframes
    for short_name, long_name in zip(sensorNamesShort, sensorNames):
        df = getattr(data, short_name)

        # Calculate the amplitude and RMS for each direction
        new_data.append(
            {
                "Accelerometer": long_name,
                "Direction": "X",
                "Amplitude": np.ptp(df.accelerationX),
                "RMS": np.sqrt(np.mean(df.accelerationX**2)),
            }
        )
        new_data.append(
            {
                "Accelerometer": long_name,
                "Direction": "Y",
                "Amplitude": np.ptp(df.accelerationY),
                "RMS": np.sqrt(np.mean(df.accelerationY**2)),
            }
        )
        new_data.append(
            {
                "Accelerometer": long_name,
                "Direction": "Z",
                "Amplitude": np.ptp(df.accelerationZ),
                "RMS": np.sqrt(np.mean(df.accelerationZ**2)),
            }
        )

    # Convert the list of data into a DataFrame
    results_df = pd.DataFrame(new_data)

    # Optionally, you might want to set a multi-level index for easier querying
    # results_df.set_index(['Accelerometer', 'Direction'], inplace=True)

    return results_df

### experiment_fft

In [ ]:
def experiment_fft(t_rot_on, t_rot_off, window_name="hamming"):
    freq = 55  # hz
    sine = lambda t: np.sin(2 * np.pi * freq * t)

    fig, ax = plt.subplots(figsize=(20, 6))
    fig.suptitle("FFT Sampling Analysis")

    def plot_fft(t):
        signal = sine(t)
        relative_time = t - t.min()
        relative_time = relative_time.round(3)
        print(np.diff(relative_time)[1:].mean())

        window = get_window(window_name, len(signal))
        windowed_signal = signal * window

        fft_signal = fft(windowed_signal)
        fft_freqs = fftfreq(
            len(windowed_signal),
            (t[1] - t[0]),
        )

        positive_freqs = fft_freqs[: len(fft_freqs) // 2]
        positive_fft = np.abs(fft_signal[: len(fft_signal) // 2])

        ax.plot(positive_freqs, positive_fft, alpha=0.5)

    plot_fft(t_rot_on)
    plot_fft(t_rot_off)
    fig.savefig("./plots/fft_sample_study.png")
    plt.show()

### frequency_analysis_side_by_side

In [ ]:
def frequency_analysis_side_by_side(
    dataset_1, dataset_2, short_name, long_name, columns, filename
):
    if dataset_1.rot_status == "off":
        data_off = getattr(dataset_1, short_name)
        data_on = getattr(dataset_2, short_name)
    else:
        data_off = getattr(dataset_2, short_name)
        data_on = getattr(dataset_1, short_name)

    start = min(data_off.index[0], data_on.index[0])
    end = max(data_off.index[-1], data_on.index[-1])

    fig, axs = plt.subplots(1, 2, figsize=(20, 6), sharey=True)
    fig.suptitle(
        f"FFT of {long_name} - Off vs. On\n"
        f"From {start.strftime('%Y-%m-%d %H:%M:%S')} to {end.strftime('%Y-%m-%d %H:%M:%S')}"
    )

    # Define a function to perform FFT and plot for a given dataset
    def plot_fft(data, ax, title):
        data["relative_time"] = data["times"] - data["times"].min()

        for i, col in enumerate(columns):
            detrended_acc = detrend(data[col])
            window = get_window("hamming", len(detrended_acc))
            windowed_acc = detrended_acc * window

            fft_acc = fft(windowed_acc)
            fft_freqs = fftfreq(
                len(windowed_acc),
                (data["relative_time"].iloc[1] - data["relative_time"].iloc[0]),
            )

            positive_freqs = fft_freqs[: len(fft_freqs) // 2]
            positive_fft = np.abs(fft_acc[: len(fft_acc) // 2])

            ax.plot(positive_freqs, positive_fft + 500 * i, label=col, alpha=0.5)

        ax.set_title(title)
        ax.set_xlabel("Frequency (Hz)")
        ax.set_ylabel("Magnitude")
        ax.grid(True)
        ax.legend()

    # Plot for 'Off' state
    plot_fft(data_off, axs[0], "Off")
    # Plot for 'On' state
    plot_fft(data_on, axs[1], "On")

    fig.tight_layout()
    fig.savefig(f"./plots/{filename}")
    plt.show()

### frequency_analysis_side_by_side_with_resample

In [ ]:
def frequency_analysis_side_by_side_with_resample(
    dataset_1, dataset_2, short_name, long_name, columns, filename
):
    if dataset_1.rot_status == "off":
        data_off = getattr(dataset_1, short_name)
        data_on = getattr(dataset_2, short_name)
    else:
        data_off = getattr(dataset_2, short_name)
        data_on = getattr(dataset_1, short_name)

    data_off = data_off.resample("5ms").nearest()
    data_on = data_on.resample("5ms").nearest()

    start = min(data_off.index[0], data_on.index[0])
    end = max(data_off.index[-1], data_on.index[-1])

    fig, axs = plt.subplots(1, 2, figsize=(20, 6), sharey=True)
    fig.suptitle(
        f"FFT of {long_name} - Off vs. On\n"
        f"From {start.strftime('%Y-%m-%d %H:%M:%S')} to {end.strftime('%Y-%m-%d %H:%M:%S')}"
    )

    # Define a function to perform FFT and plot for a given dataset
    def plot_fft(data, ax, title):
        data["relative_time"] = data["times"] - data["times"].min()
        # data["relative_time"] = (data.index - data.index[0]).to_datetime()
        print(data["relative_time"].diff()[1:].mean())

        for i, col in enumerate(columns):
            detrended_acc = detrend(data[col])
            window = get_window("hamming", len(detrended_acc))
            windowed_acc = detrended_acc * window

            fft_acc = fft(windowed_acc)
            fft_freqs = fftfreq(
                len(windowed_acc),
                (data["relative_time"].iloc[1] - data["relative_time"].iloc[0]),
            )

            positive_freqs = fft_freqs[: len(fft_freqs) // 2]
            positive_fft = np.abs(fft_acc[: len(fft_acc) // 2])

            ax.plot(positive_freqs, positive_fft + 500 * i, label=col, alpha=0.5)

        ax.set_title(title)
        ax.set_xlabel("Frequency (Hz)")
        ax.set_ylabel("Magnitude")
        ax.grid(True)
        ax.legend()

    # Plot for 'Off' state
    plot_fft(data_off, axs[0], "Off")
    # Plot for 'On' state
    plot_fft(data_on, axs[1], "On")

    fig.savefig(f"./plots/{filename}")
    plt.show()

### fft_for_rotator_data

In [ ]:
def fft_for_rotator_data(start, end, case):
    """
    Produce a plot with four axes. The two axes at the top show the Rotator
    Position and the Rotator motor torques between `start` and `end`. The
    two bottom axes show the FFT of the same telemetry.

    Parameters
    ----------
    start : str
        Timestamp in ISOT for the beginning of the telemetry.
    end : str
        Timestamp in ISOT for the end of the telemetry.
    case : str
        String used for filename and for plot title.
    """
    df_rot = getEfdData(
        client,
        "lsst.sal.MTRotator.rotation",
        columns=["actualPosition"],
        begin=start,
        end=end,
    )

    df_rot_t = getEfdData(
        client,
        "lsst.sal.MTRotator.motors",
        columns=["torque0", "torque1"],
        begin=start,
        end=end,
    )

    fig, axs = plt.subplots(2, 2, figsize=(15, 7))
    fig.suptitle(
        f"Rotator Telemetry Data - Position and Torque\n{case}\n"
        f"Start {start}, End {end}"
    )

    # Define a function to perform FFT and plot for a given dataset
    def plot_fft(data, ax, title, columns):
        data["times"] = data.index
        data["relative_time"] = data["times"] - data["times"].min()
        data["relative_time"] = data["relative_time"].dt.total_seconds()
        line_styles = ["-", "--", ":"]

        for i, col in enumerate(columns):
            detrended_acc = detrend(data[col])
            window = get_window("hamming", len(detrended_acc))
            windowed_acc = detrended_acc * window

            fft_acc = fft(windowed_acc)
            fft_freqs = fftfreq(
                len(windowed_acc),
                (data["relative_time"].iloc[1] - data["relative_time"].iloc[0]),
            )

            positive_freqs = fft_freqs[: len(fft_freqs) // 2]
            positive_fft = np.abs(fft_acc[: len(fft_acc) // 2])

            max_freq = positive_freqs[positive_fft.argmax()]

            ax.plot(positive_freqs, positive_fft, label=col, alpha=0.5)
            ax.axvline(
                max_freq,
                color=f"C{i}",
                ls=line_styles[i],
                label=f"Max freq for {col} = {max_freq:.2} Hz",
            )
            ax.grid(":", alpha=0.25)

            print(f"Max freq: {max_freq}")

    for df in [df_rot, df_rot_t]:
        df["t"] = (df.index - df.index[0]).total_seconds()

    axs[0][0].plot(
        df_rot["t"],
        df_rot.actualPosition,
        label=f"Position\n"
        f"  amplitude {np.ptp(df_rot.actualPosition):.2e} deg\n"
        f"  std {np.std(df_rot.actualPosition):.2e} deg\n",
    )
    axs[0][0].grid(":", alpha=0.25)
    axs[0][0].set_ylabel("Angle [deg]")
    axs[0][0].set_xlabel("Time [s]")
    axs[0][0].legend()

    axs[0][1].plot(
        df_rot_t["t"],
        df_rot_t.torque0 - df_rot_t.torque0.mean(),
        label=f"Torque0\n"
        f"  amplitude {np.ptp(df_rot_t.torque0):.2e} N.m\n"
        f"  std {np.std(df_rot_t.torque0):.2e} N.m",
    )
    axs[0][1].plot(
        df_rot_t["t"],
        df_rot_t.torque1 - df_rot_t.torque1.mean(),
        label=f"Torque1\n"
        f"  amplitude {np.ptp(df_rot_t.torque1):.2e} N.m\n"
        f"  std {np.std(df_rot_t.torque1):.2e} N.m",
    )
    axs[0][1].grid(":", alpha=0.25)
    axs[0][1].set_ylabel("Motors Torques [N.m]")
    axs[0][1].set_xlabel("Time [s]")
    axs[0][1].legend()

    plot_fft(df_rot, axs[1][0], "FFT Angle", ["actualPosition"])
    axs[1][0].set_ylabel("FFT Position [?]")
    axs[1][0].set_xlabel("Frequency [Hz]")
    axs[1][0].legend()

    plot_fft(df_rot_t, axs[1][1], "FFT Torque", ["torque0", "torque1"])
    axs[1][1].set_ylabel("FFT Torques [?]")
    axs[1][1].set_xlabel("Frequency [Hz]")
    axs[1][1].legend()

    fig.tight_layout()
    fig.savefig(f"plots/rotator_data_{'_'.join(case.lower().split(' '))}.png")
    plt.show()

### print_amplitudes

In [ ]:
def print_amplitudes(dataset_1, dataset_2):
    if dataset_1.rot_status == "off":
        dfs_off = dataset_1
        dfs_on = dataset_2
    else:
        dfs_off = dataset_2
        dfs_on = dataset_1

    pd.set_option("display.width", 1000)

    results_df_off = calculate_amplitudes(dfs_off)
    results_df_on = calculate_amplitudes(dfs_on)

    results_df_off["State"] = "Off"
    results_df_on["State"] = "On"

    # Merge the two DataFrames
    merged_df = pd.concat([results_df_off, results_df_on])

    # Pivot the DataFrame to better compare 'On' vs 'Off' states for each accelerometer and direction
    pivot_table = merged_df.pivot_table(
        index=["Accelerometer", "Direction"],
        columns="State",
        values=["Amplitude", "RMS"],
    )

    # Reset index if you prefer a flat structure
    pivot_table.reset_index(inplace=True)

    # Display the pivot table
    print(pivot_table)

## Data analysis from 2024-03-29

[BLOCK-197] is the test we were running when the vibration happened.  
We want to compare its execution with the Rotator PXI and drives powered on versus powered off.  
We executed both tests on the night of March 29th, 2024.  
The day obs is `20240329`. The code below grabs the scripts ran for this block.  

[BLOCK-197]: https://rubinobs.atlassian.net/browse/BLOCK-197

In [ ]:
day_obs = 20240329
block_id = 197

block_parser = BlockParser(day_obs)
blocks = block_parser.getBlockNums()
list_of_blocks = block_parser.getSeqNums(block_id)

for num in list_of_blocks:
    block = block_parser.getBlockInfo(block_id, num)
    print(
        f"Script SeqNum = {num:02} - Started at {block.begin.iso} - Completed at {block.end.iso}"
    )

Accordingly to [OLE/ROLEX on 2024-03-29], the **first** BLOCK-197 execution had the Rotator electronics **powered off**.   
The **second** execution had the Rotator electronics **powered on**. 

[OLE/ROLEX on 2024-03-29]: https://summit-lsp.lsst.codes/rolex?log_date=2024-03-29

In [ ]:
data = {
    "block197_20240329_001": {
        "day_obs": 20240329,
        "start": block_parser.getBlockInfo(block_id, 1).begin,
        "end": block_parser.getBlockInfo(block_id, 10).end,
    },
    "block197_20240329_002": {
        "day_obs": 20240329,
        "start": block_parser.getBlockInfo(block_id, 11).begin,
        "end": block_parser.getBlockInfo(block_id, 20).end,
    },
}

In [ ]:
# Join the two dictionaries and convert them into a namespace for convenience
for key, vals in data.items():
    temp_dict = query_efd_data(vals["start"], vals["end"])
    vals = temp_dict | vals
    vals = Namespace(**vals)
    data[key] = vals

### BLOCK-197 - Rotator On

In [ ]:
%matplotlib inline
key = "block197_20240329_001"
plot_dataframes(
    data[key],
    title=f"OBS-467 Investigation {data[key].day_obs} - Running BLOCK-197 w/ Rotator {data[key].rot_status}",
    filename=f"block197_rotator_{data[key].rot_status}_{data[key].day_obs}.png",
)

### BLOCK-197 - Rotator Off

In [ ]:
%matplotlib inline
key = "block197_20240329_002"
plot_dataframes(
    data[key],
    title=f"OBS-467 Investigation - Running BLOCK-197 w/ Rotator {data[key].rot_status}",
    filename=f"block197_rotator_{data[key].rot_status}_{data[key].day_obs}.png",
)

### BLOCK-197 - On/Off Comparison

The plots above shows how the Rotator and the Accelerometers respond to the hexapod movements. 
We are interested in knowing the amplitude of the vibrations and their main frequencies. 

Assuming, for simplicity, that the vibration was tangencial to the circle containing the SMRs.
For the Camera SMRs, a displacement of 2 mm in a circle of 850 mm would need the rotator to oscillate with an amplitude of 0.135 deg.
For the M2 SMRs, a displacement of 2 mm in a circle of 1740 mm would need the rotator to oscillate with an amplitude of 0.065 deg.

We could lock on M2 SMRs. Oscillations about 0.1 mm. In contrast of Cam SMRs, of 2 mm. 

In [ ]:
print_amplitudes(
    dataset_1=data["block197_20240329_001"], dataset_2=data["block197_20240329_002"]
)

We want to identify the main frequencies on each of the accelerometer data and, possibly, in other telemetry too.  
Before diving into the Fourier Transform, let's check the sampling of each data.  

In [ ]:
for key in data.keys():
    dfs = data[key]
    for attr in sensorNamesShort:
        df = getattr(dfs, attr)
        mean = df.times.diff()[1:].mean()
        std = df.times.diff()[1:].std()
        print(
            f"Rotator {dfs.rot_status} - Sampling rate is mean = {mean * 1e3:.3f} ms w/ stddev = {std * 1e3:.3f} ms"
        )

What is the effect of this high standard deviation in the data?  
Let's get the FFT of a sin function at 55 Hz and plot it with the two different samplings.

#### SST M2 Surrogate

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240329_001"],
    dataset_2=data["block197_20240329_002"],
    short_name="m2surr",
    long_name="SST M2 Surrogate",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240329_sst_m2_surrogate_fft.png",
)

#### SST spider spindle

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240329_001"],
    dataset_2=data["block197_20240329_002"],
    short_name="spider",
    long_name="SST spider spindle",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240329_sst_spider_spindle_fft.png",
)

#### SST top end ring +x -y

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240329_001"],
    dataset_2=data["block197_20240329_002"],
    short_name="ter_pxmy",
    long_name="SST top end ring +x -y",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240329_sst_top_end_ring_plusX_minusY.png",
)

#### SST top end ring -x -y

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240329_001"],
    dataset_2=data["block197_20240329_002"],
    short_name="ter_mxmy",
    long_name="SST top end ring -x -y",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240329_sst_top_end_ring_minusX_minusY.png",
)

## Data analysis from 2024-03-28

We have reports that data from 2024-03-28 are noisy too. 
Let's have a quick look at them.

In [ ]:
day_obs = 20240328
block_id = 197

block_parser = BlockParser(day_obs)
blocks = block_parser.getBlockNums()
list_of_blocks = block_parser.getSeqNums(block_id)

for num in list_of_blocks:
    block = block_parser.getBlockInfo(block_id, num)
    print(
        f"Script SeqNum = {num:02} - Started at {block.begin.iso} - Completed at {block.end.iso}"
    )

Accordingly to [OLE/ROLEX on 2024-03-28], the first BLOCK-197 execution had the Rotator electronics powered off.   
The second execution had the Rotator electronics powered on. 

[OLE/ROLEX on 2024-03-28]: https://summit-lsp.lsst.codes/rolex?log_date=2024-03-28

In [ ]:
data = {
    "block197_20240328_001": {
        "day_obs": 20240328,
        "start": block_parser.getBlockInfo(block_id, 1).begin,
        "end": block_parser.getBlockInfo(block_id, 10).end,
        "rot_status": "off",
    },
    "block197_20240328_002": {
        "day_obs": 20240328,
        "start": block_parser.getBlockInfo(block_id, 11).begin,
        "end": block_parser.getBlockInfo(block_id, 20).end,
        "rot_status": "on",
    },
}

In [ ]:
# Join the two dictionaries and convert them into a namespace for convenience
for key, vals in data.items():
    temp_dict = query_efd_data(vals["start"], vals["end"])
    vals = temp_dict | vals
    vals = Namespace(**vals)
    data[key] = vals

### BLOCK-197 - Rotator Off

In [ ]:
%matplotlib inline
key = "block197_20240328_001"
plot_dataframes(
    data[key],
    title=f"OBS-467 Investigation - Running BLOCK-197 w/ Rotator {data[key].rot_status}",
    filename=f"block197_rotator_{data[key].rot_status}_{data[key].day_obs}.png",
)

### BLOCK-197 - Rotator On

In [ ]:
%matplotlib inline
key = "block197_20240328_002"
plot_dataframes(
    data[key],
    title=f"OBS-467 Investigation - Running BLOCK-197 w/ Rotator {data[key].rot_status}",
    filename=f"block197_rotator_{data[key].rot_status}_{data[key].day_obs}.png",
)

### BLOCK-197 - On/Off Comparison

In [ ]:
print_amplitudes(
    dataset_1=data["block197_20240328_001"], dataset_2=data["block197_20240328_002"]
)

In [ ]:
for key in data.keys():
    dfs = data[key]
    for attr in sensorNamesShort:
        df = getattr(dfs, attr)
        mean = df.times.diff()[1:].mean()
        std = df.times.diff()[1:].std()
        print(
            f"Rotator {dfs.rot_status} - Sampling rate is mean = {mean * 1e3:.3f} ms w/ stddev = {std * 1e3:.3f} ms"
        )

#### SST M2 Surrogate

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240328_001"],
    dataset_2=data["block197_20240328_002"],
    short_name="m2surr",
    long_name="SST M2 Surrogate",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240328_sst_m2_surrogate_fft.png",
)

#### SST spider spindle

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240328_001"],
    dataset_2=data["block197_20240328_002"],
    short_name="spider",
    long_name="SST spider spindle",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240328_sst_spider_spindle_fft.png",
)

#### SST top end ring +x -y

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240328_001"],
    dataset_2=data["block197_20240328_002"],
    short_name="ter_pxmy",
    long_name="SST top end ring +x -y",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240328_sst_top_end_ring_plusX_minusY.png",
)

#### SST top end ring -x -y

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side(
    dataset_1=data["block197_20240328_001"],
    dataset_2=data["block197_20240328_002"],
    short_name="ter_mxmy",
    long_name="SST top end ring -x -y",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240328_sst_top_end_ring_minusX_minusY.png",
)

### BLOCK-197 - Hexapod Analysis

The plots below are two different attempts of finding criteria for a possible alarm.   
They compare the rolling std of the data with the rotator on versus rotator off. 

In [ ]:
if data["block197_20240328_001"].rot_status == "on":
    dfs_on = data["block197_20240328_001"]
    dfs_off = data["block197_20240328_002"]
else:
    dfs_on = data["block197_20240328_002"]
    dfs_off = data["block197_20240328_001"]


fig, (ax_off, ax_on) = plt.subplots(nrows=2, figsize=(10, 5), num="hexapod_analysis")

t_off = (dfs_off.camhex.index - dfs_off.camhex.index[0]).total_seconds()
t_on = (dfs_on.camhex.index - dfs_on.camhex.index[0]).total_seconds()

ax_off.plot(
    t_off,
    dfs_off.camhex.position0.rolling("500ms").std(),
    alpha=0.5,
)
ax_off.grid(":", alpha=0.5)
ax_off.set_xlabel("Time [s]")
ax_off.set_ylabel("Rolling STD [um]")
ax_off.set_title(
    f"Rotator Off - CamHex X\n"
    f"Start at {dfs_off.camhex.index[0].strftime('%Y-%m-%d %H:%M:%S')}"
)


ax_on.semilogy(
    t_on,
    dfs_on.camhex.position0.rolling("500ms").std(),
    alpha=0.5,
)
ax_on.grid(":", alpha=0.5)
ax_on.set_xlabel("Time [s]")
ax_on.set_ylabel("Rolling STD [um]")
ax_on.set_title(
    f"Rotator On - CamHex X\n"
    f"Start at {dfs_on.camhex.index[0].strftime('%Y-%m-%d %H:%M:%S')}"
)

fig.suptitle("Camera Hexapod Analysis using Rolling STDDEV")
fig.tight_layout()

## Play with FFT and Sampling

The features we see on data from 20240328 and from 20240329 are inconsistent. For 20240328, we see them when the rotator is powered on. For 20240329, we see them when the rotator is powered off. On hypothesis is that these features are associated with the data sampling. The experiment below takes the FFT of a sine function with a frequency of 55 Hz. The sampling depends on the index of the data obtained on 20240328. 

In [ ]:
experiment_fft(
    t_rot_off=data["block197_20240328_001"].m2surr.times.to_numpy(),
    t_rot_on=data["block197_20240328_002"].m2surr.times.to_numpy(),
)

The sampling seems to be slightly different when comparing the times in the first and in the second dataset. And we see similar artifacts in the FFT transform. This means that the features in the plots above are probably associated with data sampling. 

In [ ]:
%matplotlib inline
frequency_analysis_side_by_side_with_resample(
    dataset_1=data["block197_20240328_001"],
    dataset_2=data["block197_20240328_002"],
    short_name="m2surr",
    long_name="SST M2 Surrogate",
    columns=["accelerationX", "accelerationY", "accelerationZ"],
    filename=f"20240328_sst_m2_surrogate_fft_sample.png",
)

## Rotator data analysis

In [ ]:
# During vibration event --
start = Time("2024-03-10T03:00:00Z", scale="utc", format="isot")
end = Time("2024-03-10T03:01:00Z", scale="utc", format="isot")
case = "TEA Vibration"

fft_for_rotator_data(start, end, case)

In [ ]:
# Noise level ---
start = Time("2024-03-09T09:00:00Z", scale="utc", format="isot")
end = Time("2024-03-09T09:01:00Z", scale="utc", format="isot")
case = "Noise Level"

fft_for_rotator_data(start, end, case)